In [1]:
import numpy as np
import pandas as pd

from UpdateDB import Endpoint_annotation as ep


*** CompoundDB module found. Will check the synonyms table to resolve CAS. ***



In [2]:
ep = ep.Endpoint(host='dummy host', dbname='dummy db name', user='dummy user', password='dummy password')

In [3]:
heh_df = pd.read_sql_query("""SELECT subs_id, sub.class_name, sub.preferred_name, casr_number, ec_number, index_number, 
                                ht."type", hc.classification, hn."name"
                                FROM heh h
                                left join substance sub on sub.id = h.subs_id
                                left join heh_type ht on ht.id = h.heh_type_id
                                left join heh_classification hc on hc.id = h.heh_classif_id
                                left join heh_name hn on hn.id = h.heh_name_id""", ep.conn)

In [4]:
reg_df = pd.read_sql_query("""SELECT reg.id, reg.subs_id, sub.class_name, sub.preferred_name, rco.country, rt."type", rg.general_regulation_name, 
                            rspec.specific_regulation_name, rsub.subspecific_regulation_name, rsc.special_cases_name, addr.additional_information_name, 
                            reg.chem_id_name, ct."type", regn.names
                            FROM regulations reg
                            LEFT JOIN substance sub ON sub.id = reg.subs_id
                            left join regulation_country rco on rco.id = reg.reg_country_id
                            left join regulation_type rt on rt.id = reg.reg_type_id
                            left join general_regulation rg on rg.id = reg.gen_reg_id
                            left join specific_regulation rspec on rspec.id = reg.spec_reg_id
                            LEFT JOIN subspecific_regulation rsub ON rsub.id = reg.subspec_reg_id
                            left join special_cases_regulation rsc on rsc.id = reg.special_cases_id
                            left join additional_information_regulation addr on addr.id = reg.additional_information_id
                            LEFT JOIN chem_type ct ON ct.id = reg.chem_type_id
                            LEFT JOIN regulation_names regn ON regn.id = reg.regulation_id
                            order by reg.id asc""", ep.conn)

In [5]:
substances = ep.get_substances().sort_values(by='id').id.unique()

In [6]:
cmr_positive = heh_df.loc[(heh_df['type'] == 'CMR') & (~heh_df['name'].isin(['YES','NO','No information','Pending (1)',
                                                                            '(Shall apply from 1 March 2018)',
                                                                            '(31/03/2017)',
                                                                            'Registry of current Harmonised Classification and Labelling intentions by France',
                                                                            'Submitted intention by Netherlands (18/11/2015)',
                                                                            'Submitted intention by Norway (03/10/2011)',
                                                                            'Submitted intention by Germany (22/02/2011)',
                                                                            'Submitted intention by Norway (24/06/2014)',
                                                                            'Amendment 2016/1179', 'Pending','particle diameter < 1 mm)',
                                                                            'Submitted intention by Italy (22/08/2012)',
                                                                            'Submitted intention by Sweden (14/11/2014)'])), 'name'].unique()
pbt_ann = heh_df.loc[(heh_df['type'] == 'PBT') & (~heh_df['name'].isin(['YES','NO','No information','Pending (2)'])), 'name'].unique()
vpvb_ann = heh_df.loc[(heh_df['type'] == 'vPvB') & (~heh_df['name'].isin(['NO','Pending (2)','YES','No information'])), 'name'].unique()
endoc_ann = heh_df.loc[(heh_df['type'] == 'Endocrine_disruptor') & (~heh_df['name'].isin(['YES','NO','Pending (3)','No information'])), 'name'].unique()
sens_ann = heh_df.loc[(heh_df['type'] == 'Sensitiser') & (~heh_df['name'].isin(['YES','NO','No information','(Shall apply from 1 March 2018)',
                                                                                'Pending (3)','Pending(3)','Submitted intention by Slovenia (09/01/2015)',
                                                                               'Submitted intention by United Kingdom (01/02/2017)',
                                                                               'Amendment 2016/1179','(05/01/2016)','particle diameter < 1 mm)',
                                                                               'Proposed future entry in Annex VI of CLP Regulation',
                                                                               'Registry of current Harmonised Classification and Labelling intentions by Norway',
                                                                               'Removal of','SCL ≥ 0.06 %'])), 'name'].unique()
other_ann = heh_df.loc[(heh_df['type'] == 'Other') & (~heh_df['name'].isin(['No information'])), 'name'].unique()

In [7]:
pbt_vpvb_pos = reg_df[(reg_df['general_regulation_name'].isin(['pbt_vpvb'])) &
                      (~reg_df['names'].isin(['No information', 'Not PBT', 'Not vPvB', 'No information available',
                                              '(See REACH part-Candidate List section)','Conclusion: not fulfilling PBT & vPvB criteria',
                                   'Conclusion: deferred the decision on the substance from the list of potential PBT or vPvB substances.',
                                              'Conclusion: not fulfilling PBT  criteria','Conclusion: not fulfilling POP  criteria']))].names.unique()

In [8]:
endoc_pos = reg_df[(reg_df['general_regulation_name'].isin(['endocrine_disruptors']))&
                   (~reg_df['names'].isin(['Not included','No information available']))].names.unique()

In [9]:
new_cmr = ['Toxic for reproduction (Article 57c)','Toxic for reproduction (Article 57c):','Toxic for reproduction','Scope: ED, CMR','Scope: CMR',
           'Carcinogenic (Article 57a)','Carcinogenic (Category 1B)','Carcinogenic  1B','Toxic for reproduction (category 1B)',
           'Mutagenic (Article 57b)','Carcinogenic (category 1A)','Mutagenic (category 1B)',]
new_pbt_vpvb = ['PBT (Article 57 d)','Scope: vPvB (Article 57 e)', 'vPvB (Article 57 e)','Scope: PBT']
other_new = ['Equivalent level of concern having probable serious effects to environment (Article 57 f):','Scope: EQC',
             'Equivalent level of concern having probable serious effects to environment (Article 57 f)']
new_endoc = ['Scope: ED, CMR']

cmr_positive = np.concatenate((cmr_positive, new_cmr), axis=0)
pbt_vpvb_pos = np.concatenate((pbt_vpvb_pos, new_pbt_vpvb), axis=0)
endoc_pos = np.concatenate((endoc_pos,new_endoc), axis=0)
other_ann = np.concatenate((other_ann, other_new), axis=0)

In [10]:
vpvb_pos = ['vPvB substance', 'PBT/ vPvB Substance',
       'PBT/ vPvB substance', 'Rapporteur: Austria', 'Rapporteur: Norway',
       'Rapporteur: United Kingdom', 'Rapporteur: Denmark',
       'Rapporteur: Spain', 'Rapporteur: Germany',
       'Rapporteur: France',
       'Conclusion: Under evaluation', 'Rapporteur: Netherlands',
       'Rapporteur: Sweden',
       'Conclusion: fulfilling PBT & vPvB criteria',
       'Conclusion: fulfilling PBT & vPvB criteria & POP']

In [11]:
pbt_pos = ['PBT Substance', 'PBT/ vPvB Substance',
       'PBT/ vPvB substance', 'Rapporteur: Austria', 'Rapporteur: Norway',
       'Rapporteur: United Kingdom', 'Rapporteur: Denmark',
       'Rapporteur: Spain', 'Rapporteur: Germany',
       'Conclusion: Fulfilling PBT criteria & POP','Rapporteur: France',
       'Conclusion: Under evaluation', 'Rapporteur: Netherlands',
       'Conclusion: fulfilling PBT criteria', 'Rapporteur: Sweden',
       'Conclusion: fulfilling PBT & vPvB criteria',
       'Conclusion: fulfilling PBT & vPvB criteria & POP']

In [12]:
endpoint_dict = {'CMR':cmr_positive, 'PBT':pbt_pos, 'vPvB':vpvb_pos, 'Endocrine Disruptor':endoc_pos, 'Sensitiser':sens_ann}

In [13]:
subs_endpoint = ep.get_annotations_for_substance(substances, endpoint_dict)

In [14]:
subs_endpoint.head()

,subs_id,CMR,PBT,vPvB,Endocrine Disruptor,Sensitiser
0,1.0,YES,No information,No information,No information,YES
1,2.0,YES,No information,No information,YES,No information
2,3.0,YES,No information,No information,No information,No information
3,4.0,YES,No information,No information,No information,No information
4,5.0,No information,No information,No information,No information,No information


### Here it should come the check part
After getting the endpoint annotations per substance, and at the same time we update the database with them, the check function should come.
Before updating the DB, the check function should see if there's a substance id in the table.
If no, adds the new substance with the annotations.
If there is, then it checks the annotations and applies the filters we've spoken

In [15]:
# ep.add_annotations_to_database(subs_endpoint)

### Total annotation check
This only counts the total number of annotations per endpoint we have

In [16]:
ep.get_total_annotations_per_endpoint(subs_endpoint)

,Endpoints,YES,Pending,No information
0,CMR,153.0,59.0,1155.0
1,PBT,44.0,0.0,1323.0
2,vPvB,49.0,0.0,1318.0
3,Endocrine Disruptor,148.0,0.0,1219.0
4,Sensitiser,64.0,68.0,1235.0
